<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [ ]:
##### Your Code Here #####

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [2]:
data = "/home/alex/Downloads/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"

df = pd.read_csv(data)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.shape

(7043, 21)

In [4]:
df[df.columns[0:10]]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No
...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No


In [5]:
df[df.columns[11:-1]]

,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5
2,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...,...
7038,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5
7039,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9
7040,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6


In [6]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
for column in df.columns:
    print(df[column].value_counts(dropna=False), "\n\n")

3420-YJLQT    1
2208-NKVVH    1
5178-LMXOP    1
0916-QOFDP    1
4092-OFQZS    1
             ..
9995-HOTOH    1
6234-RAAPL    1
0014-BMAQU    1
5433-KYGHE    1
0302-JOIVN    1
Name: customerID, Length: 7043, dtype: int64 


Male      3555
Female    3488
Name: gender, dtype: int64 


0    5901
1    1142
Name: SeniorCitizen, dtype: int64 


No     3641
Yes    3402
Name: Partner, dtype: int64 


No     4933
Yes    2110
Name: Dependents, dtype: int64 


1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64 


Yes    6361
No      682
Name: PhoneService, dtype: int64 


No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64 


Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64 


No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64 


N

In [40]:
df[df["tenure"] == 0]["TotalCharges"].iloc[0]

''

In [42]:
df[df["tenure"] == 0] = df[df["tenure"] == 0].replace("", np.NaN)

In [43]:
df[df["tenure"] == 0]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


In [45]:
df["TotalCharges"] = df["TotalCharges"].fillna(0)

In [47]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [48]:
columns_to_drop = ["customerID"]
columns_to_one_hot_encode = ["gender","SeniorCitizen","Partner",
                             "Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup",
                             "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaymentMethod"]
columns_to_scale = ["tenure", "MonthlyCharges", "TotalCharges"]

target = "Churn"

In [49]:
encoded_columns = OneHotEncoder().fit_transform(df[columns_to_one_hot_encode]).toarray()

In [50]:
print(encoded_columns.shape, "\n\n")

encoded_columns

(7043, 41) 




array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [55]:
df[columns_to_scale].dtypes

tenure              int64
MonthlyCharges    float64
TotalCharges       object
dtype: object

In [59]:
df["TotalCharges"] = df["TotalCharges"].astype(float)

In [62]:
scaled_columns = StandardScaler().fit_transform(df[columns_to_scale])

In [63]:
df[columns_to_scale].values.shape

(7043, 3)

In [64]:
scaled_columns.shape

(7043, 3)

In [65]:
X = np.concatenate([scaled_columns, encoded_columns], axis=1)
y = df[target].values

In [73]:
print(X.shape)

print(y.shape)

(7043, 44)
(7043,)


* batch_size
* training epochs
* optimizer
* learning rate (if applicable to optimizer)
* momentum (if applicable to optimizer)
* activation functions
* network weight initialization
* dropout regularization
* number of neurons in the hidden layer

In [81]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(1, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8, 16, 32, 64, 128],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8015045404434205 using {'batch_size': 8, 'epochs': 20}
Means: 0.8015045404434205, Stdev: 0.00987181421130951 with: {'batch_size': 8, 'epochs': 20}
Means: 0.7998013019561767, Stdev: 0.01016519037266177 with: {'batch_size': 16, 'epochs': 20}
Means: 0.7996590375900269, Stdev: 0.010524668233499736 with: {'batch_size': 32, 'epochs': 20}
Means: 0.795682692527771, Stdev: 0.010289154748928464 with: {'batch_size': 64, 'epochs': 20}
Means: 0.7959666728973389, Stdev: 0.003345409803239082 with: {'batch_size': 128, 'epochs': 20}


In [82]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(1, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8, 16, 32, 64, 128]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8012219548225403 using {'batch_size': 8, 'epochs': 8}
Means: 0.8012219548225403, Stdev: 0.010050260563132242 with: {'batch_size': 8, 'epochs': 8}
Means: 0.8010787844657898, Stdev: 0.008153940516689042 with: {'batch_size': 8, 'epochs': 16}
Means: 0.8000851988792419, Stdev: 0.006606820883514515 with: {'batch_size': 8, 'epochs': 32}
Means: 0.800085473060608, Stdev: 0.008772926684642383 with: {'batch_size': 8, 'epochs': 64}
Means: 0.7998013019561767, Stdev: 0.007654808960710557 with: {'batch_size': 8, 'epochs': 128}


In [83]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(lr=0.01):
    # create model
    opt = Adam(learning_rate=lr)
    model = Sequential()
    model.add(Dense(1, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8],
             "lr": [0.001, 0.01, 0.1, 0.2]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8016474843025208 using {'batch_size': 8, 'epochs': 8, 'lr': 0.01}
Means: 0.8000855922698975, Stdev: 0.008678950129105268 with: {'batch_size': 8, 'epochs': 8, 'lr': 0.001}
Means: 0.8016474843025208, Stdev: 0.008747418195984322 with: {'batch_size': 8, 'epochs': 8, 'lr': 0.01}
Means: 0.8000847816467285, Stdev: 0.008440002405478066 with: {'batch_size': 8, 'epochs': 8, 'lr': 0.1}
Means: 0.8005094170570374, Stdev: 0.008756236742760973 with: {'batch_size': 8, 'epochs': 8, 'lr': 0.2}


In [84]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(neurons=1):
    # create model
    opt = Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(neurons, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8],
             "neurons": [1, 2, 4, 8, 16, 32, 64, 128]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8027828454971313 using {'batch_size': 8, 'epochs': 8, 'neurons': 16}
Means: 0.7999435424804687, Stdev: 0.008442336904267728 with: {'batch_size': 8, 'epochs': 8, 'neurons': 1}
Means: 0.800794494152069, Stdev: 0.00640812771115472 with: {'batch_size': 8, 'epochs': 8, 'neurons': 2}
Means: 0.8023575067520141, Stdev: 0.009453877762676317 with: {'batch_size': 8, 'epochs': 8, 'neurons': 4}
Means: 0.7985237002372741, Stdev: 0.010084998654523575 with: {'batch_size': 8, 'epochs': 8, 'neurons': 8}
Means: 0.8027828454971313, Stdev: 0.007544507897010074 with: {'batch_size': 8, 'epochs': 8, 'neurons': 16}
Means: 0.7982385039329529, Stdev: 0.00961927959131484 with: {'batch_size': 8, 'epochs': 8, 'neurons': 32}
Means: 0.7988079786300659, Stdev: 0.00604459647247513 with: {'batch_size': 8, 'epochs': 8, 'neurons': 64}
Means: 0.7959667682647705, Stdev: 0.008304958718755062 with: {'batch_size': 8, 'epochs': 8, 'neurons': 128}


In [85]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(activation_function="relu"):
    # create model
    opt = Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(16, activation=activation_function))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8],
             "activation_function": ["relu", "selu", "elu", "exponential","linear","swish"]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8027822375297546 using {'activation_function': 'elu', 'batch_size': 8, 'epochs': 8}
Means: 0.7946897745132446, Stdev: 0.008766105645484672 with: {'activation_function': 'relu', 'batch_size': 8, 'epochs': 8}
Means: 0.801220440864563, Stdev: 0.002882669504832236 with: {'activation_function': 'selu', 'batch_size': 8, 'epochs': 8}
Means: 0.8027822375297546, Stdev: 0.007371600370201635 with: {'activation_function': 'elu', 'batch_size': 8, 'epochs': 8}
Means: 0.7989489316940308, Stdev: 0.008144763334737 with: {'activation_function': 'exponential', 'batch_size': 8, 'epochs': 8}
Means: 0.798382556438446, Stdev: 0.010388818789130205 with: {'activation_function': 'linear', 'batch_size': 8, 'epochs': 8}
Means: 0.7990904569625854, Stdev: 0.00830913972363985 with: {'activation_function': 'swish', 'batch_size': 8, 'epochs': 8}


In [88]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(final_activation_function):
    # create model
    opt = Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(16, activation="elu"))
    model.add(Dense(1, activation=final_activation_function))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8],
             "final_activation_function": ["sigmoid", "tanh", "softmax", "hard_sigmoid","relu","selu"]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8019309639930725 using {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'sigmoid'}
Means: 0.8019309639930725, Stdev: 0.010324259724173257 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'sigmoid'}
Means: 0.7640196681022644, Stdev: 0.025943037338179605 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'tanh'}
Means: 0.2653713583946228, Stdev: 0.004757620185291315 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'softmax'}
Means: 0.7972452998161316, Stdev: 0.00727970538216707 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'hard_sigmoid'}
Means: 0.7565001249313354, Stdev: 0.026492377641376262 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'relu'}
Means: 0.7478352546691894, Stdev: 0.023681493337214973 with: {'batch_size': 8, 'epochs': 8, 'final_activation_function': 'selu'}


In [89]:
seed = 7
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(optimizer):
    # create model
    opt = Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(16, activation="elu"))
    model.add(Dense(1, activation="sigmoid"))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [8],
             "optimizer": ["Adadelta", "tanh", "softmax", "hard_sigmoid","relu","selu"]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7992335200309754 using {'batch_size': 8, 'epochs': 8, 'optimizer': 'softmax'}
Means: 0.7945473194122314, Stdev: 0.005722641844809545 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'Adadelta'}
Means: 0.7979556202888489, Stdev: 0.010160728280519756 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'tanh'}
Means: 0.7992335200309754, Stdev: 0.010206520625147429 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'softmax'}
Means: 0.7934106588363647, Stdev: 0.007598477174544272 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'hard_sigmoid'}
Means: 0.7908552527427674, Stdev: 0.011082280024053797 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'relu'}
Means: 0.7861687660217285, Stdev: 0.01353771852356666 with: {'batch_size': 8, 'epochs': 8, 'optimizer': 'selu'}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?